In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
# --- Load image ---
img = sitk.ReadImage('/home/abigail/synthetic_CT/910/MR_T1_TFE_1.5mm.nii', sitk.sitkFloat32)
# --- Smooth image ---
s_img = sitk.CurvatureFlow(img, timeStep=0.125, numberOfIterations=5)
# --- Threshold to roughly capture the head ---
# You may need to adjust lowerThreshold depending on your MRI contrast
mask = sitk.BinaryThreshold(s_img, lowerThreshold=5.0, upperThreshold=1000.0, insideValue=1, outsideValue=0)
# --- Fill holes in 3D ---
mask_filled = sitk.BinaryFillhole(mask, fullyConnected=True)
# --- Keep only the largest connected component (the head) ---
cc = sitk.ConnectedComponent(mask_filled)
stats = sitk.LabelShapeStatisticsImageFilter()
stats.Execute(cc)
largest_label = max(stats.GetLabels(), key=lambda l: stats.GetPhysicalSize(l))
final_mask = sitk.BinaryThreshold(cc, lowerThreshold=largest_label, upperThreshold=largest_label, insideValue=1, outsideValue=0)
# --- Convert to NumPy for interactive viewing ---
final_mask = sitk.Flip(final_mask,[False,False,True])
data = sitk.GetArrayFromImage(final_mask)
print("Mask range:", np.min(data), "to", np.max(data))
final_mask_8bit = sitk.Cast(final_mask, sitk.sitkUInt8)
# Save as .mha
sitk.WriteImage(final_mask_8bit, "/home/abigail/Synthrad2025_Task_1/Dataset/HN/910/MASK.mha")
print("8-bit mask saved as head_mask.mha")